In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from math import sin
from functools import reduce

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import ModuleList as mdl
import torch.optim as optim
import torch.nn.functional as F
import tqdm

from wxml.mlp import MLP
from wxml.train import train
from wxml.evaluate import evaluate
from wxml.data import make_loader

In [11]:
# functions to test
def parity(x):
    return x % 2


def compose(fs):
    def compose2(f, g):
        return lambda *a, **kw: f(g(*a, **kw))
    return reduce(compose2, fs)


# makes two-column dataset, first is data input to function of choice, second gets replaced w/ function output
def make_xs(n):    
    return np.random.randint(0, 10, (n, 2))


# calls function of choice, f
def make_data(n, f, dtype=None):
    xs = make_xs(n) if dtype is None else make_xs(n).astype(dtype)
    xs[:, 1] = f(xs[:, 0])
    xs, ys = xs[:, 0], xs[:, 1]
    return xs, ys


# TODO: add batches
def make_data_parity(n):
    xs_train, ys_train = make_data(n, parity)
    xs_test, ys_test = make_data(n // 10, parity)
    return xs_train, ys_train, xs_test, ys_test


def make_data_sin(n):
    xs_train, ys_train = make_data(n, np.sin, dtype=np.float32)
    xs_test, ys_test = make_data(n // 10, np.sin, dtype=np.float32)
    return xs_train, ys_train, xs_test, ys_test



def euclidean_distance(x, y):
    return torch.sqrt(torch.sum((x - y) ** 2))


def averager(f):
    return lambda x, y: f(x, y) / len(x)


n = 1000

xs_train_parity, ys_train_parity, xs_test_parity, ys_test_parity = make_data_parity(n)
xs_train_sin, ys_train_sin, xs_test_sin, ys_test_sin = make_data_sin(n)


print("parity:", list(zip(xs_train_parity[:5], ys_train_parity[:5])))

print("sin:", list(zip(xs_train_sin[:5], ys_train_sin[:5])))

parity: [(np.int64(0), np.int64(0)), (np.int64(6), np.int64(0)), (np.int64(3), np.int64(1)), (np.int64(0), np.int64(0)), (np.int64(5), np.int64(1))]
sin: [(np.float32(4.0), np.float32(-0.7568025)), (np.float32(5.0), np.float32(-0.9589243)), (np.float32(3.0), np.float32(0.14112)), (np.float32(4.0), np.float32(-0.7568025)), (np.float32(2.0), np.float32(0.9092974))]


In [12]:
lr = 1e-3

num_layers = 2
input_dim = 1
output_dim = 1
hidden_dim = 10

model = MLP(num_layers, input_dim, hidden_dim, output_dim)
modek = compose([round, model])
opt = optim.SGD(model.parameters(), lr=lr)
loader_parity = make_loader(xs_train_parity, ys_train_parity, 32)

In [13]:
epochs = 100

train(model, averager(euclidean_distance), opt, loader_parity, epochs=epochs)

loss: 0.500, acy: 0.000: 100%|██████████| 100/100 [00:07<00:00, 13.02it/s]


In [14]:
def round(x):
    if x >= 0.5: x = 1
    else: x = 0
    return x

print(xs_test_parity[:5], ys_test_parity[:5])
# Convert test data into DataLoader
loader_test_parity = make_loader(xs_test_parity, ys_test_parity, batch_size=32)

# Evaluate the model
evaluate(model, averager(euclidean_distance), loader_test_parity)


[8 6 7 1 7] [0 0 1 1 1]
tensor([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 1, 1, 0])
tensor([0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 0, 0, 1, 1, 1])
tensor([1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1])
tensor([0, 1, 0, 0])
Test Loss: 0.125, Test Accuracy: 0.550


(0.12464101552963257, 0.55)